In [ ]:
import tensorflow as tf
import sys

sys.path.append('../input/swintransformertf')

from swintransformer import SwinTransformer
import numpy as np
import pandas as pd
from tensorflow.keras import backend as K
import os
from sklearn.model_selection import StratifiedKFold
import tensorflow_addons as tfa

In [ ]:
BATCH_SIZE = 8
IMG_SIZE = 224

AUTOTUNE = tf.data.experimental.AUTOTUNE

label='Pawpularity'

lr = 2e-5
epochs = 5

SEED=999


tf.random.set_seed(SEED)

#decay_steps = 100
#decay_rate = 0.96

### Dataset Function

In [ ]:
def sample_beta_distribution(size, concentration_0=0.2, concentration_1=0.2):
    gamma_1_sample = tf.random.gamma(shape=[size], alpha=concentration_1)
    gamma_2_sample = tf.random.gamma(shape=[size], alpha=concentration_0)
    return gamma_1_sample / (gamma_1_sample + gamma_2_sample)


In [ ]:
@tf.function
def get_box(lambda_value):
    cut_rat = tf.math.sqrt(1.0 - lambda_value)

    cut_w = IMG_SIZE * cut_rat  # rw
    cut_w = tf.cast(cut_w, tf.int32)

    cut_h = IMG_SIZE * cut_rat  # rh
    cut_h = tf.cast(cut_h, tf.int32)

    cut_x = tf.random.uniform((1,), minval=0, maxval=IMG_SIZE, dtype=tf.int32)  # rx
    cut_y = tf.random.uniform((1,), minval=0, maxval=IMG_SIZE, dtype=tf.int32)  # ry

    boundaryx1 = tf.clip_by_value(cut_x[0] - cut_w // 2, 0, IMG_SIZE)
    boundaryy1 = tf.clip_by_value(cut_y[0] - cut_h // 2, 0, IMG_SIZE)
    bbx2 = tf.clip_by_value(cut_x[0] + cut_w // 2, 0, IMG_SIZE)
    bby2 = tf.clip_by_value(cut_y[0] + cut_h // 2, 0, IMG_SIZE)

    target_h = bby2 - boundaryy1
    if target_h == 0:
        target_h += 1

    target_w = bbx2 - boundaryx1
    if target_w == 0:
        target_w += 1

    return boundaryx1, boundaryy1, target_h, target_w


@tf.function
def cutmix(train_ds_one, train_ds_two):
    (image1, label1), (image2, label2) = train_ds_one, train_ds_two

    alpha = [0.25]
    beta = [0.25]

    # Get a sample from the Beta distribution
    lambda_value = sample_beta_distribution(1, alpha, beta)

    # Define Lambda
    lambda_value = lambda_value[0][0]

    # Get the bounding box offsets, heights and widths
    boundaryx1, boundaryy1, target_h, target_w = get_box(lambda_value)

    # Get a patch from the second image (`image2`)
    crop2 = tf.image.crop_to_bounding_box(
        image2, boundaryy1, boundaryx1, target_h, target_w
    )
    # Pad the `image2` patch (`crop2`) with the same offset
    image2 = tf.image.pad_to_bounding_box(
        crop2, boundaryy1, boundaryx1, IMG_SIZE, IMG_SIZE
    )
    # Get a patch from the first image (`image1`)
    crop1 = tf.image.crop_to_bounding_box(
        image1, boundaryy1, boundaryx1, target_h, target_w
    )
    # Pad the `image1` patch (`crop1`) with the same offset
    img1 = tf.image.pad_to_bounding_box(
        crop1, boundaryy1, boundaryx1, IMG_SIZE, IMG_SIZE
    )

    # Modify the first image by subtracting the patch from `image1`
    # (before applying the `image2` patch)
    image1 = image1 - img1
    # Add the modified `image1` and `image2`  together to get the CutMix image
    image = image1 + image2

    # Adjust Lambda in accordance to the pixel ration
    lambda_value = 1 - (target_w * target_h) / (IMG_SIZE * IMG_SIZE)
    lambda_value = tf.cast(lambda_value, tf.float32)

    # Combine the labels of both images
    label = lambda_value * label1 + (1 - lambda_value) * label2
    return image, label

In [ ]:
def id_to_path(img_id,dir):
    return os.path.join(dir, f'{img_id}.jpg')

def get_image(path):
    image = tf.image.decode_jpeg(tf.io.read_file(path), channels=3)
    image = tf.cast(tf.image.resize_with_pad(image, IMG_SIZE, IMG_SIZE), dtype=tf.float32)
    return image

def process_dataset(path, label):
    return get_image(path), label


def get_dataset(x,y=None,train=True):
    if train:
        y=tf.cast(y,'float32')
        ds_one = tf.data.Dataset.from_tensor_slices((x, y)).map(process_dataset, num_parallel_calls=AUTOTUNE).shuffle(256)\
            .batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
        
        ds_two = tf.data.Dataset.from_tensor_slices((x, y)).map(process_dataset, num_parallel_calls=AUTOTUNE).shuffle(256)\
            .batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
        
        train_ds = tf.data.Dataset.zip((ds_one,ds_two))
        
        return train_ds.shuffle(256).map(cutmix, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
    
    else:
        ds= tf.data.Dataset.from_tensor_slices((x, y)).map(process_dataset, num_parallel_calls=AUTOTUNE).shuffle(256)\
            .batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
        
        return ds

### Kfolds Training

#### Model

In [ ]:
#loss
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true*100 - tf.nn.sigmoid(y_pred)*100)))
    
def focal_cross_entropy(y_true,y_pred):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true,logits=y_pred))

In [ ]:
class Crossing(tf.keras.layers.Layer):
    def __init__(self,dim=None):
        super().__init__()
        self.c1=tfa.layers.PolynomialCrossing(projection_dim=dim)
        self.c2=tfa.layers.PolynomialCrossing(projection_dim=dim)
        self.c3=tfa.layers.PolynomialCrossing(projection_dim=dim)
        self.c4=tfa.layers.PolynomialCrossing(projection_dim=dim)
        self.c5=tfa.layers.PolynomialCrossing(projection_dim=dim)
        
        self.network=tf.keras.Sequential([
            tf.keras.layers.Dense(32,activation='elu'),
            tf.keras.layers.Dense(32,activation='elu'),
            tf.keras.layers.Dense(32,activation='elu'),
            tf.keras.layers.Dense(32,activation='elu'),
            tf.keras.layers.Dense(32,activation='elu'),
        ])
        
    def call(self,x,training=False):
        x1=self.c1((x,x),training=training)
        x2=self.c2((x,x1),training=training)
        x3=self.c3((x,x2),training=training)
        x4=self.c4((x,x3),training=training)
        x5=self.c5((x,x4),training=training)
        h=self.network(x,training=training)
        o=tf.concat([x5,h],axis=-1)
        return o

#### Training

In [ ]:
train_data_csv = '../input/petfinder-pawpularity-score/train.csv'
test_data_csv  = '../input/petfinder-pawpularity-score/test.csv'
train_folder = '../input/petfinder2-cropped-dataset/crop'
test_folder  = '../input/petfinder-pawpularity-score/test'

In [ ]:
data_train = pd.read_csv(train_data_csv)
data_test = pd.read_csv(test_data_csv)

data_train['path'] = data_train['Id'].apply(lambda x: id_to_path(x, train_folder))
data_test['path'] = data_test['Id'].apply(lambda x: id_to_path(x, test_folder))

In [ ]:
kfolds=5
skfold=StratifiedKFold(n_splits=kfolds,shuffle=True,random_state=SEED)

#callbacks
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=2, restore_best_weights=True)

In [ ]:
#loss
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true*100 - tf.nn.sigmoid(y_pred)*100)))
    
def cross_entropy(y_true,y_pred):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_pred, labels=y_true))


def get_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(
            tf.cast(data, tf.float32), mode="torch"), input_shape=[IMG_SIZE,IMG_SIZE,3]),
        SwinTransformer('swin_large_224', include_top=False, pretrained=True), #1536
        tf.keras.layers.Dense(32,activation='elu'),
        tf.keras.layers.Dense(1)
    ])
    
    #compile
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                    loss=cross_entropy,
                    metrics=[cross_entropy,rmse])
    return model

In [ ]:
for i,(train_idx,val_idx) in enumerate(skfold.split(data_train['path'],data_train[label])):
    print(f'fold_{i}')
    train=data_train.loc[train_idx]
    val=data_train.loc[val_idx]
    
    x_train,y_train=train['path'],train[label]/100.
    x_val,y_val=val['path'],val[label]/100.
    
    train_data=get_dataset(x_train,y_train)
    val_data=get_dataset(x_val,y_val)
    
    tf.keras.backend.clear_session()
    model=get_model()
    
    history=model.fit(train_data,validation_data=val_data,
                  epochs=epochs,verbose=1,callbacks=[early_stop],use_multiprocessing=True, workers=-1)
    
    print('save model....')
    tf.keras.models.save_model(model,f'./model_{i}')